# Memora-LangMem: Semantic Memory with Personality-Driven Thinking for LangGraph

This notebook provides a comprehensive tutorial on using `memora-langmem`, a drop-in replacement for LangGraph's standard memory stores that adds advanced semantic memory capabilities powered by Memora.

## What is Memora-LangMem?

`memora-langmem` is a Python package that implements LangGraph's `BaseStore` interface using Memora as the backend. It provides:

### Core Features

1. **Drop-in Replacement**: Fully compatible with LangGraph's memory system - just swap the store!
2. **Semantic Memory**: Advanced semantic search with spreading activation algorithms
3. **Personality-Driven Thinking**: Memory retrieval influenced by configurable agent personalities
4. **Fact Extraction**: Automatic extraction and structuring of facts from conversations
5. **Temporal Reasoning**: Time-aware memory with event date tracking
6. **Entity Linking**: Automatic recognition and linking of entities across memories
7. **Multi-Agent Support**: Each namespace can represent a different agent with unique personality traits

### What You Get vs Standard LangGraph Memory

| Feature | Standard LangGraph Memory | Memora-LangMem |
|---------|---------------------------|----------------|
| Basic Key-Value Storage | ✅ | ✅ |
| Semantic Search | ✅ (with index config) | ✅ Enhanced with spreading activation |
| Namespace Support | ✅ | ✅ |
| Personality-Driven Retrieval | ❌ | ✅ Configurable personality traits |
| Automatic Fact Extraction | ❌ | ✅ NLP-powered extraction |
| Entity Recognition | ❌ | ✅ Automatic entity linking |
| Temporal Reasoning | ❌ | ✅ Time-aware queries |
| Opinion Formation | ❌ | ✅ Agent forms opinions over time |
| Background Knowledge | ❌ | ✅ Agent-specific background context |
| Thinking/Reasoning API | ❌ | ✅ Explicit reasoning with memory |

### When to Use Memora-LangMem

- **Conversational Agents**: When you need agents to remember context across long conversations
- **Personalized AI**: When agent responses should be influenced by personality and past interactions
- **Knowledge Management**: When you need to extract and organize facts from unstructured text
- **Multi-Agent Systems**: When different agents need isolated memory with distinct personalities
- **Research & Analysis**: When you need semantic search over large knowledge bases

## Installation

```bash
# Install from local path (development)
uv pip install -e /path/to/memora-langmem

# Or with pip
pip install -e /path/to/memora-langmem
```

### Prerequisites

You need a running Memora API server. Set the URL via environment variable:

```bash
export MEMORA_API_URL=http://localhost:8000
```

## Part 1: Basic Usage - Direct Store API

Let's start with the basic `BaseStore` interface that's compatible with LangGraph.

In [ ]:
import os
from memora_langmem import MemoraStore

# Initialize the store
base_url = os.getenv("MEMORA_API_URL", "http://localhost:8000")
store = MemoraStore(base_url=base_url, default_agent_id="tutorial_agent")

print("✅ MemoraStore initialized")

### Storing and Retrieving Memories

The store uses a namespace-key-value structure:
- **Namespace**: A tuple of strings representing a hierarchical path (e.g., `("user", "alice")`)
- **Key**: A unique identifier within the namespace
- **Value**: A dictionary containing your data

In [ ]:
# Store a memory
namespace = ("user", "alice")
key = "preferences"
value = {
    "theme": "dark",
    "language": "python",
    "notifications": True,
    "interests": ["machine learning", "data science", "artificial intelligence"]
}

store.put(namespace, key, value)
print(f"✅ Stored preferences for {namespace}")

# Retrieve the memory
import time
time.sleep(1)  # Brief pause for processing

retrieved = store.get(namespace, key)
if retrieved:
    print(f"\n📦 Retrieved memory:")
    print(f"  Namespace: {retrieved.namespace}")
    print(f"  Key: {retrieved.key}")
    print(f"  Value: {retrieved.value}")
    print(f"  Created: {retrieved.created_at}")

### Semantic Search - The Power of Memora

Unlike simple key-value retrieval, Memora provides semantic search with spreading activation. This means:
- Search by natural language queries
- Find semantically related memories
- Memories are ranked by relevance

In [ ]:
# Add more memories to demonstrate search
memories = [
    ("notes", "ml_project", {
        "title": "Machine Learning Project Ideas",
        "content": "Working on a neural network for image classification. Interested in transformers and attention mechanisms."
    }),
    ("notes", "data_tools", {
        "title": "Favorite Data Tools",
        "content": "Love using pandas for data manipulation, scikit-learn for ML, and PyTorch for deep learning."
    }),
    ("notes", "meeting_summary", {
        "title": "Team Meeting Notes",
        "content": "Discussed the new AI assistant project. Team decided to use LangGraph for orchestration."
    })
]

for ns, k, v in memories:
    store.put(("user", "alice", ns), k, v)

time.sleep(2)  # Allow time for indexing

# Now search semantically
print("🔍 Searching for 'machine learning projects'...\n")
results = store.search(
    namespace_prefix=("user", "alice"),
    query="machine learning projects",
    limit=5
)

for i, result in enumerate(results, 1):
    print(f"{i}. Score: {result.score:.3f}")
    print(f"   Key: {result.key}")
    print(f"   Value: {result.value}")
    print()

### Namespace Management

Namespaces allow you to organize memories hierarchically. In Memora, each unique namespace combination maps to a separate agent.

In [ ]:
# Create memories in different namespaces
store.put(("user", "bob", "preferences"), "theme", {"theme": "light", "language": "javascript"})
store.put(("user", "charlie", "preferences"), "theme", {"theme": "auto", "language": "rust"})

time.sleep(1)

# List all namespaces
print("📁 All namespaces:")
namespaces = store.list_namespaces(prefix=("user",), limit=10)
for ns in namespaces:
    print(f"  - {ns}")

# List namespaces with specific prefix
print("\n📁 Namespaces for alice:")
alice_namespaces = store.list_namespaces(prefix=("user", "alice"), limit=10)
for ns in alice_namespaces:
    print(f"  - {ns}")

## Part 2: Integration with LangGraph Memory Tools

The real power comes from using MemoraStore with LangGraph's memory tools and agents.

In [ ]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langgraph.prebuilt import create_react_agent

# Create a fresh store for the agent
agent_store = MemoraStore(
    base_url=base_url,
    default_agent_id="intelligent_assistant"
)

# Create memory tools
manage_tool = create_manage_memory_tool(namespace=("conversations",))
search_tool = create_search_memory_tool(namespace=("conversations",))

# Create a LangGraph agent with Memora-backed memory
agent = create_react_agent(
    "anthropic:claude-3-5-sonnet-latest",
    tools=[manage_tool, search_tool],
    store=agent_store  # This is where MemoraStore plugs in!
)

print("✅ Agent created with Memora-backed memory")

### Conversational Memory in Action

Let's see how the agent uses Memora to remember information across conversations.

In [ ]:
# First conversation: Share information
print("💬 Conversation 1: Sharing preferences\n")
result1 = agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! I want you to remember some things about me:
        - My name is David
        - I'm a software engineer working on AI projects
        - I love Python and machine learning
        - I'm currently building a chatbot using LangGraph
        Please remember these details for future conversations."""
    }]
})

print("Agent response:")
print(result1["messages"][-1].content)
print("\n" + "="*80 + "\n")

# Second conversation: Recall information
time.sleep(2)  # Brief pause

print("💬 Conversation 2: Testing recall\n")
result2 = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "What do you remember about me and my work?"
    }]
})

print("Agent response:")
print(result2["messages"][-1].content)
print("\n" + "="*80 + "\n")

# Third conversation: Contextual recommendations
time.sleep(2)

print("💬 Conversation 3: Using memory for personalization\n")
result3 = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Can you suggest some relevant learning resources based on what you know about me?"
    }]
})

print("Agent response:")
print(result3["messages"][-1].content)

## Part 3: Advanced Features - Beyond Standard LangGraph

Memora provides capabilities beyond the standard BaseStore interface.

### Automatic Fact Extraction

When you store natural language content, Memora automatically:
- Extracts structured facts
- Identifies entities (people, places, concepts)
- Links related facts together
- Categorizes facts by type (world knowledge, agent actions, opinions)

In [ ]:
# Store rich natural language content
conversation_store = MemoraStore(base_url=base_url, default_agent_id="fact_extractor")

conversation_content = {
    "text": """Yesterday I met with Sarah from the marketing team. She mentioned that our new 
    product launch is scheduled for next month. The team is really excited about the AI features 
    we've built. Sarah thinks it will revolutionize how customers interact with our platform. 
    I personally believe we should focus more on user experience rather than just adding features.""",
    "context": "team meeting",
    "participants": ["self", "Sarah"]
}

conversation_store.put(
    namespace=("meetings", "2024"),
    key="marketing_sync",
    value=conversation_content
)

print("✅ Stored conversation - Memora is now extracting facts...")
print("   Behind the scenes, Memora identifies:")
print("   • Entities: Sarah, marketing team, new product, AI features")
print("   • Events: product launch next month, meeting with Sarah")
print("   • Opinions: belief about UX focus vs features")
print("   • Relationships: Sarah works in marketing")

### Temporal Reasoning

Memora is time-aware. You can query memories with temporal context.

In [ ]:
from datetime import datetime, timedelta

# Store time-sensitive information
today = datetime.now()
yesterday = today - timedelta(days=1)
last_week = today - timedelta(days=7)

events = [
    ("event_today", {"description": "Team standup meeting", "date": today.isoformat()}),
    ("event_yesterday", {"description": "Product demo", "date": yesterday.isoformat()}),
    ("event_last_week", {"description": "Sprint planning", "date": last_week.isoformat()})
]

for key, value in events:
    conversation_store.put(("events",), key, value)

time.sleep(2)

# Search with temporal context
print("🗓️  Searching for recent events...\n")
recent_events = conversation_store.search(
    namespace_prefix=("events",),
    query="meetings this week",
    limit=5
)

for event in recent_events:
    print(f"• {event.value.get('description')} - {event.value.get('date')}")

### Multi-Agent Scenarios

Each namespace combination creates a separate agent in Memora, allowing for isolated memories and distinct personalities.

In [ ]:
# Create stores for different agent personas
store_creative = MemoraStore(base_url=base_url, default_agent_id="creative_writer")
store_analyst = MemoraStore(base_url=base_url, default_agent_id="data_analyst")
store_engineer = MemoraStore(base_url=base_url, default_agent_id="software_engineer")

# Each agent has its own perspective on the same information
shared_info = {
    "topic": "New AI Feature Launch",
    "description": "We're launching an AI-powered recommendation system"
}

# Creative writer stores with focus on narrative
store_creative.put(("projects",), "ai_launch", {
    **shared_info,
    "note": "This is a revolutionary moment - we're changing how people discover content!"
})

# Data analyst stores with focus on metrics
store_analyst.put(("projects",), "ai_launch", {
    **shared_info,
    "note": "Need to track engagement metrics, conversion rates, and user retention"
})

# Software engineer stores with focus on implementation
store_engineer.put(("projects",), "ai_launch", {
    **shared_info,
    "note": "Built using transformer models, need to optimize inference latency"
})

print("✅ Created three agents with different perspectives on the same project")
print("   Each agent's memory is isolated and can develop unique personality traits")

## Part 4: Batch Operations for Performance

When working with multiple memories, batch operations are more efficient.

In [ ]:
from langgraph.store.base import PutOp, GetOp, SearchOp

batch_store = MemoraStore(base_url=base_url, default_agent_id="batch_demo")

# Batch put operations
put_ops = [
    PutOp(namespace=("docs",), key="intro", value={"title": "Introduction", "content": "Welcome to the tutorial"}),
    PutOp(namespace=("docs",), key="setup", value={"title": "Setup", "content": "Installation instructions"}),
    PutOp(namespace=("docs",), key="usage", value={"title": "Usage", "content": "How to use the API"}),
]

print("📦 Performing batch PUT operations...")
put_results = batch_store.batch(put_ops)
print(f"✅ Stored {len(put_results)} documents\n")

time.sleep(1)

# Batch get operations
get_ops = [
    GetOp(namespace=("docs",), key="intro"),
    GetOp(namespace=("docs",), key="setup"),
    GetOp(namespace=("docs",), key="usage"),
]

print("📦 Performing batch GET operations...")
get_results = batch_store.batch(get_ops)

for item in get_results:
    if item:
        print(f"  • {item.value['title']}: {item.value['content']}")

## Part 5: Cleanup

Clean up memories when they're no longer needed.

In [ ]:
# Delete specific memories
print("🗑️  Cleaning up...\n")

# Delete from the docs namespace
batch_store.delete(("docs",), "intro")
print("✅ Deleted 'intro' document")

# Verify deletion
time.sleep(1)
deleted_item = batch_store.get(("docs",), "intro")
if deleted_item is None:
    print("✅ Confirmed: document is deleted")
else:
    print("⚠️  Document still exists")

## Summary: Why Choose Memora-LangMem?

### Key Advantages

1. **Zero Code Changes**: Drop-in replacement for existing LangGraph memory stores
2. **Enhanced Intelligence**: Automatic fact extraction, entity linking, and semantic understanding
3. **Personality System**: Agents can develop unique personalities that influence memory retrieval
4. **Production Ready**: Built on robust Memora backend with proper persistence
5. **Rich Context**: Beyond simple key-value, stores temporal, relational, and semantic information
6. **Research-Backed**: Implements spreading activation and advanced memory retrieval algorithms

### Use Cases

- **Customer Support Bots**: Remember customer preferences, history, and context
- **Personal Assistants**: Build agents that truly understand and remember user preferences
- **Knowledge Workers**: Agents that accumulate domain expertise over time
- **Research Assistants**: Semantic search over large knowledge bases
- **Team Collaboration**: Multiple agents with distinct roles and memories

### Getting Started

1. Install: `uv pip install -e /path/to/memora-langmem`
2. Start Memora API: Ensure server is running at `http://localhost:8000`
3. Replace store: `store = MemoraStore(base_url=base_url)`
4. Use normally: All LangGraph memory APIs work as expected
5. Enjoy enhanced memory capabilities automatically!

### Next Steps

- Explore the Memora API documentation for advanced features
- Configure agent personalities for different use cases
- Experiment with the thinking/reasoning API
- Build multi-agent systems with isolated memories
- Integrate with your existing LangGraph applications